In [11]:
from selenium import webdriver
import time
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time
import datetime

In [12]:
chromedriver = 'C:/chromedriver_win32/chromedriver'
driver = webdriver.Chrome(chromedriver)

## 검색한 롤 아이디 랭킹,티어,점수,승리,패배,승률,모스트1 2 3 챔피언 정보 수집

### 배치완료가 안된 아이디 검색 X (랭킹,티어가 없어서 안됨)

In [418]:
rank =[]
tier =[]
points =[]
win =[]
lose =[]
win_ratio=[]
most1=[]
most2=[]
most3=[]

cha_name=[]
data =[]
number = int(input("검색할 아이디 개수: "))
for i in range(number):
    name = input('아이디: ')
    cha_name.append(name)
    driver.get('https://op.gg')
    a = driver.find_element_by_name('userName')
    a.send_keys(name)
    a.submit()
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    rank.append(soup.select('span.ranking')[0].text)
    tier.append(soup.select('div.TierRank')[0].text)

    lp = soup.select('span.LeaguePoints')[0].text
    lp = lp.split(" ")[0]
    lp = lp.replace('\t','').replace('\n','').replace(',','')
    points.append(int(lp))
    win.append(soup.select('span.wins')[0].text)
    lose.append(soup.select('span.losses')[0].text)
    win_ratio.append(soup.select('span.winratio')[0].text.split(" ")[1])
    most1.append(soup.select('div.ChampionName > a')[0].text.replace('\n','').replace('\t',''))
    most2.append(soup.select('div.ChampionName > a')[1].text.replace('\n','').replace('\t',''))
    most3.append(soup.select('div.ChampionName > a')[2].text.replace('\n','').replace('\t',''))
data=[rank,tier,points,win,lose,win_ratio,most1,most2,most3]
data1 = np.transpose(data)
df = pd.DataFrame(data1,index=cha_name,columns=['랭킹','티어','점수','승리','패배','승률','모스트1','모스트2','모스트3'])
df['랭킹'] = df['랭킹'].str.replace(',','').astype('int64')
df.sort_values('랭킹')

검색할 아이디 개수: 3
아이디: 맵리딩
아이디: 구름동동
아이디: 민뚜비


,랭킹,티어,점수,승리,패배,승률,모스트1,모스트2,모스트3
민뚜비,565731,Gold 3,61,31승,39패,44%,레넥톤,나서스,카이사
맵리딩,1155354,Silver 1,43,13승,3패,81%,애니,그라가스,오리아나
구름동동,1344419,Silver 2,73,104승,118패,47%,제라스,세라핀,노틸러스


## 현재 각 라인별 챔피언 순위 수집


In [426]:
top=[]
jug=[]
mid=[]
ad=[]
sup=[]

driver.get('https://op.gg')
driver.find_element_by_xpath('/html/body/div[2]/div[1]/ul/li[2]/a').click()
driver.find_element_by_xpath('/html/body/div[2]/div[2]/div[2]/div[1]/div[5]/div[1]/ul/li[2]/a').click()
html = driver.page_source
soup2 =BeautifulSoup(html,'html.parser')

t = soup2.select_one('tbody.tabItem.champion-trend-tier-TOP')
top_c = t.select('div.champion-index-table__name')
for i in range(len(top_c)):
    top.append(top_c[i].text)
    
j = soup2.select_one('tbody.tabItem.champion-trend-tier-JUNGLE')
jug_c = j.select('div.champion-index-table__name')
for i in range(len(jug_c)):
    jug.append(jug_c[i].text)
    
m = soup2.select_one('tbody.tabItem.champion-trend-tier-MID')
mid_c = m.select('div.champion-index-table__name')
for i in range(len(mid_c)):
    mid.append(mid_c[i].text)  
    
a = soup2.select_one('tbody.tabItem.champion-trend-tier-ADC')
ad_c = a.select('div.champion-index-table__name')
for i in range(len(ad_c)):
    ad.append(ad_c[i].text)
    
s = soup2.select_one('tbody.tabItem.champion-trend-tier-SUPPORT')
sup_c = s.select('div.champion-index-table__name')
for i in range(len(sup_c)):
    sup.append(sup_c[i].text)
    
line=[top,jug,mid,ad,sup]
df2 = pd.DataFrame(line).T
df2.columns=['TOP','JUG','MID','AD','SUP']
df2.index = range(1,len(df2)+1)
df2

,TOP,JUG,MID,AD,SUP
1,나르,우디르,탈론,카이사,쓰레쉬
2,말파이트,엘리스,카타리나,세라핀,레오나
3,다리우스,카서스,제드,징크스,세라핀
4,레넥톤,렉사이,갈리오,시비르,렐
5,뽀삐,샤코,애니비아,베인,블리츠크랭크
6,카밀,비에고,피즈,이즈리얼,세나
7,아트록스,이블린,판테온,케이틀린,알리스타
8,피오라,피들스틱,비에고,진,마오카이
9,볼리베어,람머스,야스오,트리스타나,샤코
10,하이머딩거,누누와 윌럼프,트위스티드 페이트,세나,모르가나


## 유저의 kda와 승률의 상관관계

In [18]:
name = input('아이디: ')
driver.get('https://op.gg')
a = driver.find_element_by_name('userName')
a.send_keys(name)
a.submit()
driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[5]/div[2]/div[2]/div/div[1]/div/ul/li[2]/a').click()
time.sleep(3)


for i in range(4,10):
    driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[5]/div[2]/div[2]/div/div[2]/div[{}]/a'.format(i)).click()
    time.sleep(5)

html = driver.page_source
soup = BeautifulSoup(html,'html.parser')
times = []
win_lose =[]
kda =[]
print(soup.select('div.GameItemWrap'))
for i in soup.select('div.GameItemWrap'):
    times.append(i.select('span._timeago._timeCountAssigned.tip')[0]['data-datetime'])

day = list(map(lambda x: datetime.datetime.fromtimestamp(int(x)).strftime('%Y-%m-%d'),times))
day2 = list(map(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'),day))

for i in soup.select('div.GameResult'):
    win_lose.append(i.text.replace('\n','').replace('\t',''))
    
for i in soup.select('div.KDA > div.KDA'):
    if i.select('span.Death')[0].text == '0':
        kda.append(round(((int(i.select('span.Kill')[0].text) + int(i.select('span.Assist')[0].text)) * 1.2),2))
    else:
        kda.append(round((int(i.select('span.Kill')[0].text) + int(i.select('span.Assist')[0].text)) / int(i.select('span.Death')[0].text),2))
a =[day,win_lose]
b =[day,kda]
df = pd.DataFrame(a).T
df.columns = ['날짜','승패']
df2 = pd.DataFrame(b).T
df2.columns=['날짜','KDA']
df['Count'] = 0
df = df.groupby(['날짜','승패'],as_index=False).count()
a = df2.groupby('날짜').sum()
a['count'] = df2.groupby('날짜').count()
a['kda'] = round(a['KDA'] / a['count'],2)

win_ratio =[]
for i in df['날짜'].drop_duplicates():
    
    y = df[df['날짜'] == i]
    print(y)
    if '승리' in y['승패'].values and '패배' in y['승패'].values:
        win_ratio.append(y[y['승패']=='승리'].Count.sum() / y.Count.sum() * 100)
    else:
        if '승리' in y['승패'].values:
            win_ratio.append(100)
        else:
            win_ratio.append(0)


a['win_rate'] = win_ratio
df5 = a[['kda','win_rate']]
df5.index = list(map(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d'),df5.index))
df5

import plotly.graph_objects as go 
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y":True}]])

fig.add_trace(
    go.Scatter(x=df5.index,y=df5.kda,name='KDA'),
    secondary_y=False)
fig.add_trace(
    go.Scatter(x=df5.index,y=df5.win_rate,name='WIN_RATE'),
    secondary_y=True)

fig.update_layout(
    title_text="{}님의 kda/winrate".format(name))



아이디: 구름동동
[<div class="GameItemWrap">
<div class="GameItem Lose" data-game-id="5025599070" data-game-result="lose" data-game-time="1614086180" data-summoner-id="66071938">
<div class="Content">
<div class="GameStats">
<div class="GameType" title="솔랭">
					솔랭
				</div>
<div class="TimeStamp"><span class="_timeago _timeCountAssigned tip" data-datetime="1614086180" data-interval="60" data-type="" title="2021년 2월 23일 오후 10시 16분">19시간 전</span></div>
<div class="Bar"></div>
<div class="GameResult">
											패배									</div>
<div class="GameLength">31분 12초</div>
</div>
<div class="GameSettingInfo">
<div class="ChampionImage">
<a href="/champion/xerath/statistics" target="_blank"><img alt="제라스" class="Image" src="//opgg-static.akamaized.net/images/lol/champion/Xerath.png?image=c_scale,q_auto,w_46&amp;v=1613643960"/></a>
</div>
<div class="SummonerSpell">
<div class="Spell">
<img alt="방어막" class="Image tip" src="//opgg-static.akamaized.net/images/lol/spell/SummonerBarrier.png?image=c_scale

           날짜  승패  Count
0  2021-01-23  패배      1
           날짜  승패  Count
1  2021-01-24  승리      2
2  2021-01-24  패배     11
           날짜  승패  Count
3  2021-01-25  승리      3
4  2021-01-25  패배      3
           날짜  승패  Count
5  2021-01-26  승리      2
6  2021-01-26  패배      3
           날짜  승패  Count
7  2021-01-27  패배      2
           날짜  승패  Count
8  2021-01-28  승리      3
9  2021-01-28  패배      1
            날짜  승패  Count
10  2021-01-30  승리      5
11  2021-01-30  패배      4
            날짜  승패  Count
12  2021-01-31  승리      2
13  2021-01-31  패배      2
            날짜  승패  Count
14  2021-02-02  승리      5
15  2021-02-02  패배      2
            날짜  승패  Count
16  2021-02-03  패배      1
            날짜  승패  Count
17  2021-02-04  승리      2
18  2021-02-04  패배      1
            날짜  승패  Count
19  2021-02-05  승리      2
20  2021-02-05  패배      2
            날짜  승패  Count
21  2021-02-07  승리      3
22  2021-02-07  패배      2
            날짜  승패  Count
23  2021-02-08  승리      1
            날짜  승패  Count
24

In [16]:
df5.corr()

,kda,win_rate
kda,1.000000,0.627573
win_rate,0.627573,1.000000


In [13]:
name = input('아이디: ')
driver.get('https://op.gg')
a = driver.find_element_by_name('userName')
a.send_keys(name)
a.submit()
time.sleep(3)
driver.find_element_by_xpath('/html/body/div[2]/div[2]/div/div/div[5]/div[2]/div[2]/div/div[1]/div/ul/li[2]/a').click()
html2 = driver.page_source
soup2 = BeautifulSoup(html2,'html.parser')

아이디: 구름동동


In [10]:
a =[]
for i in soup.select('div.GameItemList'):
    print(i)
    a.append(i.select('div.Content > div.GameSettingInfo > div.ChampionName > a').text)
    print(i.select('div.Content'))
# len(a)
# a

In [26]:
soup.select('div.Content>div.GameSettingInfo>div.ChampionName > a')

[]

In [5]:
soup.select('div.GameItemWrap')

[]

In [21]:
for i in soup.select('div.GameItemWrap'):
    print(i)

In [29]:
soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div.GameItemList > div:nth-child(1)')

[]

In [4]:
soup.select('div.Content > div.GameAverageStatsBox')

[]

In [7]:
soup.select('dev.GameItemList')

[]

In [8]:
soup.select('#SummonerLayoutContent > div.tabItem.Content.SummonerLayoutContent.summonerLayout-summary > div.RealContent > div > div.Content > div:nth-child(5)')

[<div class="GameItemList">
 <div class="GameItemWrap">
 <div class="GameItem Lose" data-game-id="5025599070" data-game-result="lose" data-game-time="1614086180" data-summoner-id="66071938">
 <div class="Content">
 <div class="GameStats">
 <div class="GameType" title="솔랭">
 					솔랭
 				</div>
 <div class="TimeStamp"><span class="_timeago _timeCountAssigned tip" data-datetime="1614086180" data-interval="60" data-type="" title="2021년 2월 23일 오후 10시 16분">19시간 전</span></div>
 <div class="Bar"></div>
 <div class="GameResult">
 											패배									</div>
 <div class="GameLength">31분 12초</div>
 </div>
 <div class="GameSettingInfo">
 <div class="ChampionImage">
 <a href="/champion/xerath/statistics" target="_blank"><img alt="제라스" class="Image" src="//opgg-static.akamaized.net/images/lol/champion/Xerath.png?image=c_scale,q_auto,w_46&amp;v=1613643960"/></a>
 </div>
 <div class="SummonerSpell">
 <div class="Spell">
 <img alt="방어막" class="Image tip" src="//opgg-static.akamaized.net/images/lol/sp

In [17]:
soup2.select('div.GameItemWrap')

[]